In [ ]:
import os
import csv
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import bisect
from PIL import Image
import shutil
import random

examples_per_class = 50
num_classes = 2000

In [ ]:
df = pd.read_csv('/home/kylecshan/data/train.csv')

In [ ]:
df = df.sort_values('landmark_id')

In [ ]:
#threshold = 100
counts = df.landmark_id.value_counts()
counts = pd.DataFrame(counts)

In [ ]:
#threshold_cts = counts.loc[counts.landmark_id >= threshold, :]
#threshold_cts.columns = ['count']
#threshold_cts
threshold_cts = counts.iloc[:num_classes,:]
threshold_cts.columns = ['count']
threshold_cts.shape

In [ ]:
def mkdir_if_not_exist(path):
    if not os.path.exists(path):
        os.mkdir(path)

In [ ]:
i = 0
base_dir = '/home/kylecshan/data/images224/train_ms2000_v4/'
imgs_dir = '/home/kylecshan/data/images224/all/'
trainvalindex = (0.7, 0.9) # boundary points
problems = []

mkdir_if_not_exist(base_dir)
mkdir_if_not_exist(base_dir + 'train/')
mkdir_if_not_exist(base_dir + 'val/')
mkdir_if_not_exist(base_dir + 'index/')

for row in threshold_cts.itertuples():
    i += 1
    if i <= 0: # to resume from previous
        continue
    
    # Find indexes for this landmark
    landmark, count = row.Index, row.count
    start = df.landmark_id.searchsorted(landmark, side='left')
    end = df.landmark_id.searchsorted(landmark, side='right')
    
    # Randomly shuffle indexes for imgs of this landmark
    np.random.seed(321)
    idx_shuffle = np.random.permutation(range(start, end))
    
    # Create folder
    train_dir = base_dir + 'train/' + str(landmark) + '/'
    val_dir = base_dir + 'val/' + str(landmark) + '/'
    index_dir = base_dir + 'index/' + str(landmark) + '/'
    for d in [train_dir, val_dir, index_dir]:
        mkdir_if_not_exist(d)
    
    num_to_copy = count // 6 #examples_per_class 
    
    # Copy files over
    num_copied = 0;
    num_seen = 0;
    while num_copied < num_to_copy and num_seen < count:
        imgname = df.id.iloc[idx_shuffle[num_seen]]
        img_path = imgs_dir + imgname + '.jpg'
        try:
            im = Image.open(img_path)
            w,h = im.size
            if w >= 128 and h >= 128:
                if num_copied < num_to_copy * trainvaltest[0]:
                    shutil.copy(img_path, train_dir)
                elif num_copied < num_to_copy * trainvaltest[1]:
                    shutil.copy(img_path, val_dir)
                else:
                    shutil.copy(img_path, index_dir)
                num_copied += 1
        except:
            pass # couldn't find image, oh well
        num_seen += 1
        
    print('landmark %d (%d / %d): copied %d out of %d total' % 
          (landmark, i, threshold_cts.size, num_copied, count))
    if num_copied < num_to_copy:
        problems.append(landmark)
    #if i > 0:
    #    break

In [ ]:
problems

In [ ]:
df[start:(start+10)]

In [ ]:
idx_shuffle

In [ ]:
plt.plot(range(203094), counts)
plt.yscale('log')
plt.xscale('log')
plt.xlabel('Sorted Classes')
plt.ylabel('Count')
plt.title('Number of Examples per Class')

In [ ]:
np.sum(counts.landmark_id < 10)

In [ ]:
len(counts)